In [1]:
#!/usr/bin/python3
# import tensorflow as tf
import numpy as np
from math import exp
from numpy.random import binomial,shuffle,seed, choice
import pandas as pd
from IPython.display import display, clear_output
import os
import numpy
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import CompactLSM

seed(4) # for replicating results

Reservoir Dimension set

In [2]:
nx = 5
ny = 5
nz = 5
N = nx*ny*nz                                          # Reservoir size

N_read = 10                                           # No. of Readout neurons

Important constants related to LIF neuron and synaptic model

In [3]:
global vrest, vth, t_refrac
vrest, vth, t_refrac = 0, 20, 2

tau_m = 32
params_potential = {'C':1, 'g_L':1/tau_m, 'E_L':vrest, 'V_T':vth, 'R_p':t_refrac}

Delay = 1 #constant delay for all synapses in ms

tau_c = 64
C_theta = 5
del_C = 3
n_bits = 8
delta_c = 1
params_conc = {'C_theta':C_theta, 'del_C':del_C, 'tau_c':64, 'nbits':n_bits, 'delta_c':delta_c}

syn_string = "first-order"

sampling_freq = 12.5 # in khz
h = 1/sampling_freq # in ms

time_params = {'h':h, 'Delay':Delay}

Set Reservoir Connections

In [4]:
# Storing the IDs of the neurons
LSM_ID = np.zeros((nx,ny,nz),dtype=np.int64)
l = 0
for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            LSM_ID[i,j,k] = l
            l = l + 1

# Storing the synapse connections, and creating the initial weight matrix

k_prob = [0.45, 0.3, 0.6, 0.15]
r_sq = 2**2

W_arr = [3, 6, -2, -2]
W_init = 3
Weights_temp = np.zeros((N,N))

N_in = int(N*0.8)
neuron_type = [ int(i<N_in) for i in range(N)]
seed(4)
shuffle(neuron_type) # 1 for excitatory, 0 for inhibitory

synapes = [dict() for i in range(N)]    # an array of dictonaries which store the location of neuron, 
                                        # type of neuron, and the IDs of the neurons it is connected to

for l in range(N):
    loc = CompactLSM.ID_to_ind(nx,ny,nz,l)
    n_type = neuron_type[l]
    cons = []
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if l != int(LSM_ID[i,j,k]):
                    dist_sq = (loc[0]-i)**2 + (loc[1]-j)**2 + (loc[2]-k)**2
                    k_probl = 0
                    if n_type == 1:
                      if neuron_type[int(LSM_ID[i,j,k])] == 1:
                        k_probl = k_prob[0]
                        W_init = W_arr[0]
                      else:
                        k_probl = k_prob[1]
                        W_init = W_arr[1]
                    else:
                      if neuron_type[int(LSM_ID[i,j,k])] == 1:
                        k_probl = k_prob[2]
                        W_init = W_arr[2]
                      else:
                        k_probl = k_prob[3]
                        W_init = W_arr[3]

                    probability = k_probl* exp(-1*dist_sq/r_sq)
#                     print(probability)
                    check = binomial(1,probability)
                    if check == 1:
                        cons.append(int(LSM_ID[i,j,k]))
                        Weights_temp[l,int(LSM_ID[i,j,k])] = W_init    
    synapes[l] = {"Location":loc, "Neuron_type":n_type, "connections":cons}

global Weights
Weights = Weights_temp
print("Total synapse:", len(np.argwhere(Weights!=0)), 
        ",E --> E :", len(np.argwhere(Weights==3)), 
        ",E --> I:",len(np.argwhere(Weights==6)),
        ",I --> E/I:",len(np.argwhere(Weights==-2)))
i = 64
print("Total Connections: for neuron {}:{}, {}".format(i,synapes[i]["Neuron_type"],synapes[i]["connections"]) )

Total synapse: 1130 ,E --> E : 769 ,E --> I: 105 ,I --> E/I: 256
Total Connections: for neuron 64:1, [19, 62, 83, 84, 89, 93, 97, 109, 114, 118]


In [5]:
def show_3d_connections(net_shape, camera=(45,-135), show_what='both', show_discc=False, show_conn=False, show_index=False, show_coord=False, show_arrows=False, markersize=(10,10,70,70), figsize=(100,100)):
    '''
    Prints the neurons and the connections between them.
    net_shape->the same used to generate the liquid (list or tuple)
    show_what='both','exc' or 'inh'->controls which types of neurons should be plotted
    show_discc=False->doesn't show the neurons without connections to other ones
    show_conn=False->shows or not the lines between the connected neurons
    show_number=True->shows the index number of each neuron according to the Neurongroup
    show_coord=True->shows the x,y,z coordinates
    show_arrows=True->shows arrows indicating the direction of connections (very slow!)
    figsize=(width,height)->in millimetres
    markersize=(inh_dot,exc_dot,inh_star,exc_star)->sizes of the markers
    camera=(45,45)->camera angles, in degrees
    '''
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.pyplot as plt
    marker_size_inh,marker_size_exc,marker_size_rec_inh,marker_size_rec_exc = markersize
    user_elev,user_azim = camera
    fig_w,fig_h = figsize #figure size in millimetres
    NofN = net_shape[0]*net_shape[1]*net_shape[2]

    # This is how the liquid neurons index are distributed inside the 3D shape
    # Each line is a neuron index and the collumns are its x,y and z positions.
    count_3d=0;
    Neuron3DMatrix = numpy.empty((NofN,3),dtype=numpy.int)
    for zi in range(net_shape[2]):
        for yi in range(net_shape[1]):
            for xi in range(net_shape[0]):
                Neuron3DMatrix[count_3d][0]=xi;
                Neuron3DMatrix[count_3d][1]=yi;
                Neuron3DMatrix[count_3d][2]=zi;
                count_3d+=1;

    _exc_indices = []
    _inh_indices = []
    
    _disconnected = []
    if not show_discc:
        _disconnected = numpy.array(range(NofN))[output_connected(NofN)==0];
    
    if (show_what=='both' or show_what=='inh'):
        _inh_indices = output_inh_indices(NoINHN)
        # Positions of the neurons in the 3D space
        x_inh=[Neuron3DMatrix[i][0] for i in range(NofN) if (i in _inh_indices) and (i not in _disconnected)] 
        y_inh=[Neuron3DMatrix[i][1] for i in range(NofN) if (i in _inh_indices) and (i not in _disconnected)] 
        z_inh=[Neuron3DMatrix[i][2] for i in range(NofN) if (i in _inh_indices) and (i not in _disconnected)] 
        
        recurrent_connections_idx_inh = numpy.array(output_pre_i_connections(NoINHC)[output_pre_i_connections(NoINHC)==output_pos_i_connections(NoINHC)])
        x_rec_inh=[i[0] for i in Neuron3DMatrix[[i for i in recurrent_connections_idx_inh if i not in _disconnected]]] 
        y_rec_inh=[i[1] for i in Neuron3DMatrix[[i for i in recurrent_connections_idx_inh if i not in _disconnected]]]
        z_rec_inh=[i[2] for i in Neuron3DMatrix[[i for i in recurrent_connections_idx_inh if i not in _disconnected]]]

        # List of tuples with (pre synaptic, post synaptic) INHIBITORY->ANYTHING neurons connections indexes
        inh_connect_positions_pre  = [Neuron3DMatrix[i] for i,j in zip(output_pre_i_connections(NoINHC),range(NoINHC)) if  (output_pre_i_connections(NoINHC)[j]!=output_pos_i_connections(NoINHC)[j])]
        inh_connect_positions_post = [Neuron3DMatrix[i] for i,j in zip(output_pos_i_connections(NoINHC),range(NoINHC)) if  (output_pre_i_connections(NoINHC)[j]!=output_pos_i_connections(NoINHC)[j])]        
        
        
    if (show_what=='both' or show_what=='exc'):
        _exc_indices = output_exc_indices(NoEXCN)
        x_exc=[Neuron3DMatrix[i][0] for i in range(NofN) if (i in _exc_indices) and not (i in _disconnected)] 
        y_exc=[Neuron3DMatrix[i][1] for i in range(NofN) if (i in _exc_indices) and not (i in _disconnected)] 
        z_exc=[Neuron3DMatrix[i][2] for i in range(NofN) if (i in _exc_indices) and not (i in _disconnected)] 

        recurrent_connections_idx_exc = numpy.array(output_pre_e_connections(NoEXCC)[output_pre_e_connections(NoEXCC)==output_pos_e_connections(NoEXCC)])
        x_rec_exc=[i[0] for i in Neuron3DMatrix[[i for i in recurrent_connections_idx_exc if i not in _disconnected]]] 
        y_rec_exc=[i[1] for i in Neuron3DMatrix[[i for i in recurrent_connections_idx_exc if i not in _disconnected]]]
        z_rec_exc=[i[2] for i in Neuron3DMatrix[[i for i in recurrent_connections_idx_exc if i not in _disconnected]]]

        # List of tuples with (pre synaptic, post synaptic) EXCITATORY->ANYTHING neurons connections indexes
        exc_connect_positions_pre  = [Neuron3DMatrix[i] for i,j in zip(output_pre_e_connections(NoEXCC),range(NoEXCC)) if (output_pre_e_connections(NoEXCC)[j]!=output_pos_e_connections(NoEXCC)[j])]
        exc_connect_positions_post = [Neuron3DMatrix[i] for i,j in zip(output_pos_e_connections(NoEXCC),range(NoEXCC)) if (output_pre_e_connections(NoEXCC)[j]!=output_pos_e_connections(NoEXCC)[j])]


    fig = plt.figure() # creates the figure for the following plots
    fig.set_size_inches(fig_w/25.4,fig_h/25.4, forward=False) #Set the figure size in inches (1in == 2.54cm)
    ax = fig.add_subplot(111, projection='3d') # setup to only one

    if (show_what=='both' or show_what=='inh'):    
        ax.scatter(x_inh, y_inh, z_inh, c='b', s=[marker_size_inh]*len(x_inh)) # plots the points correnponding to the inhibitory neurons

    if (show_what=='both' or show_what=='exc'):         
        ax.scatter(x_exc, y_exc, z_exc, c='r', s=[marker_size_exc]*len(x_exc)) # plots the points correnponding to the excitatory neurons

    if (show_what=='both' or show_what=='inh'):    
        ax.scatter(x_rec_inh, y_rec_inh, z_rec_inh, c='b', marker='*', s=[marker_size_rec_inh]*len(x_rec_inh)) # plots where a inhibitory neuron has a reccurent connection

    if (show_what=='both' or show_what=='exc'):    
        ax.scatter(x_rec_exc, y_rec_exc, z_rec_exc, c='r', marker='*', s=[marker_size_rec_exc]*len(x_rec_exc)) # plots where a excitatory neuron has a reccurent connection
    
    _what_to_show = []
    if (show_what=='both' or show_what=='exc'):
        _what_to_show += _exc_indices.tolist()
    if (show_what=='both' or show_what=='inh'):
        _what_to_show += _inh_indices.tolist()        
        
    # Insert a label with the position of each neuron according to the positions_list (NeuronGroup)
    if show_index:    
        for t,n in [(Neuron3DMatrix[i],i) for i in range(NofN) if (i in _what_to_show) and (i not in _disconnected)]:
            ax.text(t[0], t[1], t[2], "["+str(n)+"]")

    # Insert a label with the 3D coordinate used to calculate the connection probabilities
    if show_coord:
        for t,n in [(Neuron3DMatrix[i],i) for i in range(NofN) if (i in _what_to_show) and (i not in _disconnected)]:
            ax.text(t[0], t[1], t[2], str(t)+"="+str(n)) # to insert also the coordinates of the point

    #
    # Draw a 3D vector (arrow)
    from matplotlib.patches import FancyArrowPatch
    from mpl_toolkits.mplot3d import proj3d

    class Arrow3D(FancyArrowPatch):
        def __init__(self, xs, ys, zs, *args, **kwargs):
            FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
            self._verts3d = xs, ys, zs

        def draw(self, renderer):
            xs3d, ys3d, zs3d = self._verts3d
            xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
            self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
            FancyArrowPatch.draw(self, renderer)

    if show_conn:
        if show_arrows:
            # Plot the EXCITATORY connections
            if (show_what=='both' or show_what=='exc'):    
                for i in range(len(exc_connect_positions_pre)):
                    a = Arrow3D(
                        [ exc_connect_positions_pre[i][0], exc_connect_positions_post[i][0] ], 
                        [ exc_connect_positions_pre[i][1], exc_connect_positions_post[i][1] ], 
                        [ exc_connect_positions_pre[i][2], exc_connect_positions_post[i][2] ], 
                        label='excitatory connections', mutation_scale=20, lw=1, arrowstyle="-|>", color="r")
                    ax.add_artist(a)
        else:
            if (show_what=='both' or show_what=='exc'):    
                # Plot the EXCITATORY connections - FAST VERSION WITHOUT ARROWS
                for i in range(len(exc_connect_positions_pre)):
                    ax.plot(
                        [ exc_connect_positions_pre[i][0], exc_connect_positions_post[i][0] ], 
                        [ exc_connect_positions_pre[i][1], exc_connect_positions_post[i][1] ], 
                        [ exc_connect_positions_pre[i][2], exc_connect_positions_post[i][2] ], 
                        label='excitatory connections', color='#FF0000')

        if show_arrows:
            if (show_what=='both' or show_what=='inh'):    
                # Plot the INHIBITORY connections
                for i in range(len(inh_connect_positions_pre)):
                    a = Arrow3D(
                        [ inh_connect_positions_pre[i][0], inh_connect_positions_post[i][0] ], 
                        [ inh_connect_positions_pre[i][1], inh_connect_positions_post[i][1] ], 
                        [ inh_connect_positions_pre[i][2], inh_connect_positions_post[i][2] ], 
                        label='inhibitory connections', mutation_scale=20, lw=1, arrowstyle="-|>", color="b")
                    ax.add_artist(a)
        else:
            if (show_what=='both' or show_what=='inh'):                
                # Plot the INHIBITORY connections - FAST VERSION WITHOUT ARROWS
                for i in range(len(inh_connect_positions_pre)):
                    ax.plot(
                        [ inh_connect_positions_pre[i][0], inh_connect_positions_post[i][0] ], 
                        [ inh_connect_positions_pre[i][1], inh_connect_positions_post[i][1] ], 
                        [ inh_connect_positions_pre[i][2], inh_connect_positions_post[i][2] ], 
                        label='inhibitory connections', color='#0000FF')


    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    plt.title("Liquid's 3D structure")
    
    ax.view_init(elev=user_elev, azim=user_azim)
    
    plt.show()

In [6]:
show_3d_connections((nx,ny,nz), camera=(45,-135), show_what='both', show_discc=False, show_conn=True, 
                    show_index=False, show_coord=False, show_arrows=False, markersize=(10,10,120,120),
                    figsize=(500,300))

NameError: name 'output_connected' is not defined

Set Readout neuron initial Weights

In [7]:
All_labels = [str(x) for x in range(10)]
# N_read = 10                                           # No. of Readout neurons
W_init_read = 1                                       # Initial weight, equal for all, update with learning
Weights_temp_readOut = W_init_read*np.ones((N_read, N), dtype='float')


synapes_read = []        # an array of dictonaries which store the label of neuron, 
                                                      # and the IDs of the neurons it is connected to

for l in range(N_read):
    label = All_labels[l]
    synapes_read.append(label)

Weights_readOut = Weights_temp_readOut

In [8]:
# creating file location and label arrays for train and validate

base = 'PreProcessing/trainBSA'
os.listdir(base)

train_Labels = []
file_name_List = []

for human in os.listdir(base):
    base_up = base + '/' + human
    for train_sample in os.listdir(base_up):
        train_Label = train_sample[0:2]
        file_loc = base_up + '/' + train_sample
        file_name_List.append(file_loc)
        train_Labels.append(train_Label)

seedval = 4

seed(seedval)
shuffle(train_Labels)
seed(seedval)
shuffle(file_name_List)

total_size = len(train_Labels)
train_size = int(total_size*0.8)

validate_Labels = train_Labels[train_size:total_size]
validate_file_list = file_name_List[train_size:total_size]

train_Labels = train_Labels[0:train_size]
train_file_list = file_name_List[0:train_size]

In [9]:
L = 78
Fin = 4
reservoir_ID = [i for i in range(N)]
seed(seedval)
Input_CXNs = choice(reservoir_ID, size = (L,Fin))
sign_win_matrix = (binomial(1,1/2, size = (L, Fin)) - 0.5)*2
# Input_CXNs * sign_win_matrix

### solving the reservoir